In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import optuna

# Cargar los datos
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

# Preprocesamiento: Separación de CPU, GPU y Memoria
def process_cpu(cpu):
    parts = cpu.split()
    return parts[0], parts[2], float(parts[-1].replace('GHz', ''))

def process_gpu(gpu):
    parts = gpu.split()
    return parts[0], ' '.join(parts[1:])

def process_memory(memory):
    types = ['SSD', 'HDD', 'Flash', 'Hybrid']
    for t in types:
        if t in memory:
            numeric_part = ''.join(filter(str.isdigit, memory))  # Extrae solo los dígitos
            return int(numeric_part), t
    numeric_part = ''.join(filter(str.isdigit, memory))
    return int(numeric_part) if numeric_part else 0, "Unknown"

# Aplicar transformación
train_df[['Cpu_Brand', 'Cpu_Family', 'Cpu_Speed']] = train_df['Cpu'].apply(lambda x: pd.Series(process_cpu(x)))
test_df[['Cpu_Brand', 'Cpu_Family', 'Cpu_Speed']] = test_df['Cpu'].apply(lambda x: pd.Series(process_cpu(x)))

train_df[['Gpu_Brand', 'Gpu_Model']] = train_df['Gpu'].apply(lambda x: pd.Series(process_gpu(x)))
test_df[['Gpu_Brand', 'Gpu_Model']] = test_df['Gpu'].apply(lambda x: pd.Series(process_gpu(x)))

train_df[['Memory_Capacity', 'Memory_Type']] = train_df['Memory'].apply(lambda x: pd.Series(process_memory(x)))
test_df[['Memory_Capacity', 'Memory_Type']] = test_df['Memory'].apply(lambda x: pd.Series(process_memory(x)))

# Convertir Ram y Weight a valores numéricos
train_df['Ram'] = train_df['Ram'].apply(lambda x: int(''.join(filter(str.isdigit, str(x)))))
test_df['Ram'] = test_df['Ram'].apply(lambda x: int(''.join(filter(str.isdigit, str(x)))))

train_df['Weight'] = train_df['Weight'].apply(lambda x: float(str(x).replace('kg', '').strip()))
test_df['Weight'] = test_df['Weight'].apply(lambda x: float(str(x).replace('kg', '').strip()))

# Codificar variables categóricas con categorías alineadas
categorical_cols = ['Company', 'TypeName', 'OpSys', 'Cpu_Brand', 'Cpu_Family', 'Gpu_Brand', 'Gpu_Model', 'Memory_Type']
for col in categorical_cols:
    train_df[col] = train_df[col].astype("category").cat.codes
    test_df[col] = test_df[col].map(train_df[col].to_dict()).fillna(-1).astype(int)

# Selección de características
X = train_df.drop(columns=['Price_in_euros', 'Cpu', 'Gpu', 'Memory', 'ScreenResolution', 'Product'])
y = train_df['Price_in_euros']
X_test = test_df.drop(columns=['Cpu', 'Gpu', 'Memory', 'ScreenResolution', 'Product'])

# Dividir en train y validación con shuffle
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Optimización de hiperparámetros con Optuna
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'random_seed': 42,
        'verbose': 0
    }
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=0)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print(f'Validation RMSE: {rmse}')
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Entrenar modelo final
best_params = study.best_params
model = CatBoostRegressor(**best_params)
model.fit(X, y, verbose=100)

# Calcular RMSE final en conjunto de validación
y_val_pred = model.predict(X_val)
final_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
print(f'Final Validation RMSE: {final_rmse}')

# Predecir precios en test.csv
test_df['Price_in_euros'] = model.predict(X_test)
test_df[['laptop_ID', 'Price_in_euros']].to_csv("submission.csv", index=False)




[I 2025-02-10 15:52:03,023] A new study created in memory with name: no-name-4d83d6ad-e35b-4341-a85a-a51ac18b3623
c:\Users\juanmoreno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-10 15:52:03,994] Trial 0 finished with value: 330.2160478300763 and parameters: {'iterations': 1949, 'depth': 8, 'learning_rate': 0.23010246519193533, 'l2_leaf_reg': 3.7969574160521935}. Best is trial 0 with value: 330.2160478300763.


Validation RMSE: 330.2160478300763


c:\Users\juanmoreno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-10 15:52:04,705] Trial 1 finished with value: 295.85004065173797 and parameters: {'iterations': 591, 'depth': 7, 'learning_rate': 0.17764380674484131, 'l2_leaf_reg': 5.863803929157864}. Best is trial 1 with value: 295.85004065173797.


Validation RMSE: 295.85004065173797


c:\Users\juanmoreno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-10 15:52:06,159] Trial 2 finished with value: 302.12125753530296 and parameters: {'iterations': 1609, 'depth': 6, 'learning_rate': 0.022107264040416755, 'l2_leaf_reg': 1.8041666664353961}. Best is trial 1 with value: 295.85004065173797.


...

c:\Users\juanmoreno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-10 15:52:51,806] Trial 48 finished with value: 294.51479261550145 and parameters: {'iterations': 1809, 'depth': 5, 'learning_rate': 0.15190092986722023, 'l2_leaf_reg': 9.895164914964605}. Best is trial 43 with value: 281.8754941710709.


Validation RMSE: 294.51479261550145


c:\Users\juanmoreno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-10 15:52:52,229] Trial 49 finished with value: 282.6037976714553 and parameters: {'iterations': 1673, 'depth': 4, 'learning_rate': 0.20778265752621128, 'l2_leaf_reg': 9.55875455129166}. Best is trial 43 with value: 281.8754941710709.


Validation RMSE: 282.6037976714553
0:	learn: 618.4417323	total: 1.07ms	remaining: 1.88s
100:	learn: 209.9910951	total: 74.5ms	remaining: 1.22s
200:	learn: 175.9140168	total: 143ms	remaining: 1.11s
300:	learn: 150.7448356	total: 215ms	remaining: 1.04s
400:	learn: 129.2872958	total: 291ms	remaining: 986ms
500:	learn: 115.6836719	total: 377ms	remaining: 949ms
600:	learn: 105.0506885	total: 452ms	remaining: 873ms
700:	learn: 96.6706314	total: 518ms	remaining: 784ms
800:	learn: 89.1149574	total: 583ms	remaining: 699ms
900:	learn: 82.3654421	total: 675ms	remaining: 645ms
1000:	learn: 76.1633634	total: 751ms	remaining: 571ms
1100:	learn: 70.8755972	total: 824ms	remaining: 495ms
1200:	learn: 66.4307606	total: 905ms	remaining: 423ms
1300:	learn: 62.3018416	total: 975ms	remaining: 346ms
1400:	learn: 58.6931942	total: 1.05s	remaining: 271ms
1500:	learn: 55.5308728	total: 1.12s	remaining: 195ms
1600:	learn: 52.4756794	total: 1.19s	remaining: 120ms
1700:	learn: 49.7525590	total: 1.27s	remaining: 45

c:\Users\juanmoreno\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
